In [1]:
def takedata_index(csv , x):
    import random
    from tqdm import tqdm_notebook as tqdm
    
    plan = csv.plans.values
    click = csv.click_mode.values

    transport_mode = []
    index = []
    
    #100% means all data
    if(x == 100):
        csv.reset_index(drop=True, inplace=True) 
        return csv.index
    
    #put the most recommend plan in to transport_mode
    for i in range(len(plan)): 
        #it must have better way  but now i just want to quickly distingush the dif 
        #if mode = 10
        if(str(str(plan[i]).split('transport_mode')[1])[4] == str(0)):
            transport_mode.append(str(10))
        #if mode = 11
        elif(str(str(plan[i]).split('transport_mode')[1])[4] == str(1)):
            transport_mode.append(str(11))
        else:
            transport_mode.append(str(str(plan[i]).split('transport_mode')[1])[3])
    
        
    #find dif index between transport_mode and click_mode
    for i in range(len(click)): 
        if(int(transport_mode[i]) != click[i]):
            index.append(i)
    
    #put most recommend_mode people __% to training
    dif_id_len = len(index)
    normal_id_len = int(dif_id_len* (x/100))
    for i in tqdm(range(normal_id_len)):
        num = random.randint(0,len(plan)-1)
        while(num in index):
            num = random.randint(0,len(plan)-1)
        index.append(num)
    index.sort()
    
    return index

In [2]:
def train_data(file, index):
    import pandas as pd
    from sklearn import preprocessing
    from sklearn.preprocessing import MinMaxScaler
    
    train_d = file.iloc[index]
    train_label = train_d.click_mode
    #choose what feature you don't need
    train_d = train_d.drop(columns= ['sid','pid','plans','req_time','plan_time','o','d','click_mode','click_time'])
    #one hot 'time_hour'
    df2 = pd.get_dummies(train_d['time_hour'])
    train_d = train_d.join(df2)
    train_d = train_d.drop(columns = ['time_hour'])
    train_d.reset_index(drop=True, inplace=True)
    
    #normalize
    scaler = MinMaxScaler()
    train_d = scaler.fit_transform(train_d)
    train_d = pd.DataFrame(train_d)
    
    return train_d, train_label

In [3]:
def rf(data, label):
    from sklearn import preprocessing, metrics, ensemble
    from sklearn.model_selection import train_test_split
    from sklearn.externals import joblib #save model
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import f1_score

    # parameters for GridSearchCV
    param_grid = {"n_estimators": [100, 200,300],
#                   "max_depth": [5,7,10],
#                   "min_samples_split": [2,5,10],
#                   "min_samples_leaf": [1, 5, 10],
#                   "max_leaf_nodes": [20, 50, 100],
                  "min_weight_fraction_leaf": [0, 0.1]}
   
    
    train_X, test_X, train_y, test_y = train_test_split(data, label, test_size = 0.3)
    #grid search
    forest = ensemble.RandomForestClassifier()
    grid_search = GridSearchCV(estimator = forest,param_grid = param_grid, n_jobs = 1, verbose=3, cv = 5)    
    grid_search.fit(train_X, train_y)
    print(grid_search.best_params_)
    #train randomforest
    forest = ensemble.RandomForestClassifier(**grid_search.best_params_)
    forest_fit = forest.fit(train_X, train_y)
    
    test_y_predicted = forest.predict(test_X)

    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    f1 = f1_score(test_y, test_y_predicted, average='weighted')
    print(forest_fit)
    print(accuracy)
    print(f1)
    joblib.dump(forest, 'rf5.pkl')

In [4]:
def xgboost(data, label):
    from sklearn import preprocessing, cross_validation, metrics, ensemble
    from sklearn.externals import joblib #save model
    from xgboost.sklearn import XGBClassifier
    from sklearn.metrics import f1_score
    
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(data, label, test_size = 0.3)
    xgbc = XGBClassifier(silent=1)
    xgbc_fit = xgbc.fit(train_X, train_y)
    test_y_predicted = xgbc.predict(test_X)

    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    f1 = f1_score(test_y, test_y_predicted, average='weighted')
    print(xgbc_fit)
    print(accuracy)
    print(f1)
    joblib.dump(xgbc, 'xgbc.pkl')

In [5]:
def main():
    import pandas as pd
    import numpy as np
    import csv
    
    train_final = pd.read_csv('/storage/kdd/train_final2.csv')
    
    index = takedata_index(train_final ,100)
    train_d, train_label = train_data(train_final, index)
    rf(train_d, train_label)
#     xgboost(train_d, train_label)

In [6]:
if __name__ == '__main__':
    main()

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] n_estimators=100, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=100, min_weight_fraction_leaf=0, score=0.696370052936728, total= 1.7min
[CV] n_estimators=100, min_weight_fraction_leaf=0 ....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[CV]  n_estimators=100, min_weight_fraction_leaf=0, score=0.6960563424664009, total= 1.7min
[CV] n_estimators=100, min_weight_fraction_leaf=0 ....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.8min remaining:    0.0s


[CV]  n_estimators=100, min_weight_fraction_leaf=0, score=0.6935839662181326, total= 1.7min
[CV] n_estimators=100, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=100, min_weight_fraction_leaf=0, score=0.6969510753958875, total= 1.7min
[CV] n_estimators=100, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=100, min_weight_fraction_leaf=0, score=0.697445755794715, total= 1.7min
[CV] n_estimators=200, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=200, min_weight_fraction_leaf=0, score=0.6967639274010587, total= 3.4min
[CV] n_estimators=200, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=200, min_weight_fraction_leaf=0, score=0.6983566780633065, total= 3.4min
[CV] n_estimators=200, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=200, min_weight_fraction_leaf=0, score=0.6959474364226514, total= 3.4min
[CV] n_estimators=200, min_weight_fraction_leaf=0 ....................
[CV]  n_estimators=200,

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 64.4min finished


{'n_estimators': 300, 'min_weight_fraction_leaf': 0}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.6991051536385762
0.6569326222586562
